# Process USURDB Data
- Goal: Take downloaded csv of data from the U.S. Utility Rate Database and convert it into a tabular format of month-hour prices

## Input Data
- The rate name is a string that describes the rate and its code
- The energy rate is broken into:
     - 12 periods, each of which has up to:
        - 4 tiers, each of which has several components:
            - rate: the electric rate
            - adjustments: fees, riders, and fuel adjustments
            - sell: rate at which electricity is sold back to the utility
            - max: the maximum kWh at which the tier applies

## Transformation Steps
- [x] Use regex to extract the rate code from the rate name and put in a separate Columns
- [x] Create an id table for each rate that identifies some of the characteristics of each rate:
    - [x] Full rate description
    - [x] Add columns with binary variables identifying whether a rate is a TOU
    - [x] Update the "sector" column to include Ag and Pump categories
- [ ] Create a rectangular dataframe of month-hour rates
    - [x] First fill in with the period number
    - [x] Then replace with the total cost for that period
        - [x] Need to calculate total cost by adding rate + adj 
    - [ ] Create this table for demand charges as well

        
## To figure out
- [ ] Decide how to handle tiers
- [ ] We might want to remove rates for primary/transmission voltage

## Output Data
- Table of rate info
- Table of month-hour energy rates
- Table of month-hour demand charges
- Aggregations of rates



In [265]:
import pandas as pd
import numpy as np
import ast
import re
import plotly.express as px

df = pd.read_csv('Inputs/USURDB_CA.csv', parse_dates=True, index_col='label')

#drop any columns with all missing data
df = df.dropna(axis=1, how='all')

df.head(2)

,eiaid,name,is_default,startdate,enddate,utility,sector,description,source,sourceparent,...,energyratestructure/period14/tier0rate,energyratestructure/period14/tier0adj,energyweekdayschedule,energyweekendschedule,energyattrs,energycomments,supersedes,coincidentrateunit,coincidentratestructure/period0/tier0rate,coincidentschedule
label,,,,,,,,,,,,,,,,,,,,,
539f6a0bec4f024411ec8b47,17609,Time-of-use - General Service - Demand Metered...,NaN,1/1/2014,12/31/2015,Southern California Edison Co,Commercial,#NAME?,http://www.sce.com/NR/sc3/tm2/pdf/CE281.pdf,NaN,...,NaN,NaN,"[[0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 1L, 1L, 1L, ...","[[0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, ...",NaN,NaN,NaN,NaN,NaN,NaN
5553987d5457a3c2378b4568,14328,E-20 Maximum demand of (1000 KW or more) (Seco...,NaN,3/1/2015,3/23/2016,Pacific Gas & Electric Co,Commercial,An optional meter data access charge is applic...,http://www.pge.com/nots/rates/tariffs/rateinfo...,http://www.pge.com/tariffs/ERS.SHTML,...,NaN,NaN,"[[0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 1L, 1L, 1L, ...","[[0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, ...",NaN,"Period 1: Off Peak Winter, Period 2: Part Peak...",5400ad765257a3fb3199cb99,NaN,NaN,NaN


## Update descriptive data for rate
- rate code
- sector
- rate type

In [267]:
rate_table = df[['name','utility','sector','voltagecategory','startdate','enddate','flatdemandstructure/period0/tier0rate','demandratestructure/period0/tier0rate']]

#get the rate code
rate_table['rate code'] = rate_table['name'].str.extract(r'((\b)([A-Z0-9]+)((-| )([A-Z0-9]+))+(\b)|^([A-Z0-9]+)\b)', expand=False)[0]
rate_table['rate code'] = rate_table['rate code'].fillna(rate_table['name'].str.extract(r'([A-Z]{3,})($|(?=[\)]))', expand=False)[0])

#get the rate option 
rate_table['rate option'] = rate_table['name'].str.extract(r'((([O|o]ption( |,)+))([A-Z0-9]+))', expand=False)[0]
rate_table['rate option'] = rate_table['rate option'].fillna(rate_table['name'].str.extract(r'(?<=[\(| ])(?<![k|0-9|\-])([A-Z])($|(?=[\)]))', expand=False)[0])
rate_table['rate option'] = rate_table['rate option'].str.replace('Option ','')

#update the sector name
rate_table.loc[rate_table['name'].str.contains('agr', case=False),'sector'] = 'Agricultural'
rate_table.loc[rate_table['name'].str.contains('pump', case=False),'sector'] = 'Pumping'

#create a column identifying if the rate is a TOU rate
rate_table['TOU'] = 0
rate_table.loc[rate_table['name'].str.contains('TOU', case=False),'TOU'] = 1
rate_table.loc[rate_table['name'].str.contains('time', case=False),'TOU'] = 1

#Determine whether the rate includes a demand charge by identifying whether it has a non-zero demand charge rate
rate_table['Demand Charge'] = (rate_table['flatdemandstructure/period0/tier0rate'].fillna(0) + rate_table['demandratestructure/period0/tier0rate'].fillna(0))
rate_table.loc[rate_table['Demand Charge'] > 0,'Demand Charge'] = 1
rate_table['Demand Charge'] = rate_table['Demand Charge'].astype('Int64')
rate_table = rate_table.drop(columns=['flatdemandstructure/period0/tier0rate','demandratestructure/period0/tier0rate'])

#create a column for the year
rate_table['Year'] = (pd.DatetimeIndex(rate_table['enddate']).year.astype('Int64'))
rate_table['Year'] = rate_table['Year'].fillna(2020)
rate_table = rate_table.drop(columns=['startdate','enddate'])

#replace the utility name with its acronym
utility_dict = {'Southern California Edison Co':'SCE',
                'Pacific Gas & Electric Co':'PGE',
                'Sacramento Municipal Utility District':'SMUD',
                'San Diego Gas & Electric Co':'SDGE',
                'Los Angeles Department of Water & Power':'LADWP'}
rate_table['utility'] = rate_table['utility'].replace(utility_dict)

rate_table.head(10)

,name,utility,sector,voltagecategory,rate code,rate option,TOU,Demand Charge,Year
label,,,,,,,,,
539f6a0bec4f024411ec8b47,Time-of-use - General Service - Demand Metered...,SCE,Commercial,Primary,TOU-GS-3,R,1,1,2015
5553987d5457a3c2378b4568,E-20 Maximum demand of (1000 KW or more) (Seco...,PGE,Commercial,Secondary,E-20,NaN,0,1,2016
539f6a8eec4f024411ec912f,Time-Of-Use Agricultural and Pumping - Large: ...,SCE,Pumping,NaN,TOU-PA-3,A,1,1,2015
539f6a8fec4f024411ec9173,Time-of-use - General Service - Demand Metered...,SCE,Commercial,Transmission,TOU-GS-3,R,1,1,2015
57b608055457a38c44ded3d3,TOU-A-P Secondary,SDGE,Commercial,Secondary,TOU-A-P,NaN,1,0,2016
539f6aeaec4f024411ec9613,"Time of Use, General Service, Demand Metered, ...",SCE,Commercial,Transmission,GS-2 TOU B,B,1,1,2015
539f6b07ec4f024411ec979d,Time-Of-Use Agricultural and Pumping-Small to ...,SCE,Pumping,NaN,TOU-PA-2,CPP,1,1,2015
5cbf88395457a30817671087,Large Agriculture Power TOU Rate (AG-5A),PGE,Agricultural,NaN,AG-5A,NaN,1,1,2020
539f6b25ec4f024411ec98e7,"Time of Use, General Service, Demand Metered, ...",SCE,Commercial,Primary,GS-2 TOU B,B,1,1,2015


## Transform to rectangular
- For each row:
 - [x] get the weekday and weekend schedules
    - [x] sum rate and adj to get the total rate for each period
    - [x] create a dictionary of period:total rate
    - [x] replace the period numbers with the rate using the dictionary



### Calculate total energy rate by period

In [269]:
#create a new df that only contains the energy rate structure columns
energy_structure = df.loc[:,df.columns.str.contains('energyratestructure/')]

# create a multiindex from these columns
energy_structure.columns = pd.MultiIndex.from_tuples([tuple([int(i) for i in re.findall(r'[0-9]+', col)] + re.findall(r'([a-z]+)$', col)) for col in energy_structure.columns], names=['Period', 'Tier','Component'])

#only keep the values for tier 0 (let's ignore higher tiers for now)
energy_structure = energy_structure.loc[:,energy_structure.columns.get_level_values(1)==0].fillna(0)
#drop the tier level of the multiindex
energy_structure.columns = energy_structure.columns.droplevel(1)


#calculate the total rate by adding rate and adj
for p in range(15):
    energy_structure[p,'total'] = energy_structure[p,'rate'] + energy_structure[p,'adj']

#only keep the total data
energy_structure = energy_structure.loc[:,energy_structure.columns.get_level_values(1)=='total']
energy_structure.columns = energy_structure.columns.droplevel(1)

energy_structure

Period,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
label,,,,,,,,,,,,,,,
539f6a0bec4f024411ec8b47,0.07252,0.09152,0.06798,0.13729,0.33658,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5553987d5457a3c2378b4568,0.07431,0.09636,0.07311,0.10275,0.14772,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539f6a8eec4f024411ec912f,0.05786,0.07496,0.05276,0.07110,0.11484,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539f6a8fec4f024411ec9173,0.07008,0.08908,0.06554,0.13485,0.33414,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57b608055457a38c44ded3d3,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5cd307b85457a3fe7154e9d4,0.11913,0.13529,0.11484,0.13433,0.14040,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581a2a9f5457a3ac3f4715b5,0.11573,0.10836,0.08756,0.10936,0.10936,0.09133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581a308a5457a38b524715b5,0.11271,0.10635,0.08619,0.10741,0.10741,0.09065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### create month-hour rate table

In [271]:
#create a blank dataframe to hold the data
energy_schedule = pd.DataFrame()

#for each row (rate)
for index, row in df.iterrows():
    months = [1,2,3,4,5,6,7,8,9,10,11,12]
    try:
        #create a dictionary of periods:rates, ignoring any columns where the rate = 0. If all rates = 0, this will raise an indexerror, and we will skip this rate
        period_to_rate_dict = energy_structure[energy_structure.index == index].replace(0,np.NaN).dropna(axis=1).to_dict(orient='records')[0]

        #get the weekday data, replacing the period numbers with the rates
        df_weekday = pd.DataFrame(data=ast.literal_eval(row['energyweekdayschedule'].replace('L','')), 
                                  index=pd.MultiIndex.from_product([[index], months, ['weekday']], 
                                  names=['label','month','day'])
                                  ).replace(period_to_rate_dict)
        #get the weekend data, replacing the period numbers with the rates
        df_weekend = pd.DataFrame(data=ast.literal_eval(row['energyweekendschedule'].replace('L','')), 
                                  index=pd.MultiIndex.from_product([[index], months, ['weekend']], 
                                  names=['label','month','day'])
                                  ).replace(period_to_rate_dict)
        #concatenate the data together
        energy_schedule = pd.concat([energy_schedule,df_weekday,df_weekend], axis=0)

    #if all of the rates = 0, it raises an error. These data will not be included in our final dataframe
    except IndexError:
        pass
    

energy_schedule

0        1        2        3   \
label                    month day                                           
539f6a0bec4f024411ec8b47 1     weekday  0.07252  0.07252  0.07252  0.07252   
                         2     weekday  0.07252  0.07252  0.07252  0.07252   
                         3     weekday  0.07252  0.07252  0.07252  0.07252   
                         4     weekday  0.07252  0.07252  0.07252  0.07252   
                         5     weekday  0.07252  0.07252  0.07252  0.07252   
...                                         ...      ...      ...      ...   
581a346d5457a3e7544715b5 8     weekend  0.08619  0.08619  0.08619  0.08619   
                         9     weekend  0.08619  0.08619  0.08619  0.08619   
                         10    weekend  0.09065  0.09065  0.09065  0.09065   
                         11    weekend  0.09065  0.09065  0.09065  0.09065   
                         12    weekend  0.09065  0.09065  0.09065  0.09065   

                                             4        5        6        7   \
label                    month day                                           
539f6a0bec4f024411ec8b47 1     weekday  0.07252  0.07252  0.07252  0.07252   
                         2     weekday  0.07252  0.07252  0.07252  0.07252   
                         3     weekday  0.07252  0.07252  0.07252  0.07252   
                         4     weekday  0.07252  0.07252  0.07252  0.07252   
                         5     weekday  0.07252  0.07252  0.07252  0.07252   
...                                         ...      ...      ...      ...   
581a346d5457a3e7544715b5 8     weekend  0.08619  0.08619  0.08619  0.08619   
                         9     weekend  0.08619  0.08619  0.08619  0.08619   
                         10    weekend  0.09065  0.09065  0.09065  0.09065   
                         11    weekend  0.09065  0.09065  0.09065  0.09065   
                         12    weekend  0.09065  0.09065  0.09065  0.09065   

                                             8        9   ...       14  \
label                    month day                        ...            
539f6a0bec4f024411ec8b47 1     weekday  0.09152  0.09152  ...  0.09152   
                         2     weekday  0.09152  0.09152  ...  0.09152   
                         3     weekday  0.09152  0.09152  ...  0.09152   
                         4     weekday  0.09152  0.09152  ...  0.09152   
                         5     weekday  0.09152  0.09152  ...  0.09152   
...                                         ...      ...  ...      ...   
581a346d5457a3e7544715b5 8     weekend  0.08619  0.08619  ...  0.08619   
                         9     weekend  0.08619  0.08619  ...  0.08619   
                         10    weekend  0.09065  0.09065  ...  0.09065   
                         11    weekend  0.09065  0.09065  ...  0.09065   
                         12    weekend  0.09065  0.09065  ...  0.09065   

                                             15       16       17       18  \
label                    month day                                           
539f6a0bec4f024411ec8b47 1     weekday  0.09152  0.09152  0.09152  0.09152   
                         2     weekday  0.09152  0.09152  0.09152  0.09152   
                         3     weekday  0.09152  0.09152  0.09152  0.09152   
                         4     weekday  0.09152  0.09152  0.09152  0.09152   
                         5     weekday  0.09152  0.09152  0.09152  0.09152   
...                                         ...      ...      ...      ...   
581a346d5457a3e7544715b5 8     weekend  0.08619  0.08619  0.08619  0.08619   
                         9     weekend  0.08619  0.08619  0.08619  0.08619   
                         10    weekend  0.09065  0.09065  0.09065  0.09065   
                         11    weekend  0.09065  0.09065  0.09065  0.09065   
                         12    weekend  0.09065  0.09065  0.09065  0.09065   

                                     

# Look up and aggregate rates
- We now have a table of rate characteristics, and a table of the month-hour rates. We want to be able to average multiple rates together to get an aggregate

## Steps
- [x] search rate table to get a list of index labels that match criteria
- [x] create a subset of the energy_schedule dataframe that only inlcudes these labels
- [x] group the data and get descriptive statistics
- [x] plot the results

In [273]:
# Search Criteria
################
utility = ['SCE'] #SCE, SDGE, PGE, SMUD, LADWP
sector = ['Pumping','Agricultural'] #Commercial, Industrial, Pumping, Agriculture
tou = [1] #either 0 or 1
years = range(2010,2021) #list years to include

In [274]:
#get a list of index labels from the rate table that match these criteria
matched_indexes = rate_table[(rate_table['utility'].isin(utility)) & 
                             (rate_table['sector'].isin(sector)) &
                             (rate_table['TOU'].isin(tou)) &
                             (rate_table['Year'].isin(years))].index.to_list()

#get the rates that match the search criteria
results = energy_schedule[energy_schedule.index.get_level_values(0).isin(matched_indexes)]
#get descriptive statistics for these rates
results = results.reset_index().drop(columns=['label']).groupby(['month','day']).describe()

#only keep certain stats
stats_to_keep = ['min','25%','50%','75%','max']
results = results.loc[:,results.columns.get_level_values(1).isin(stats_to_keep)]

#melt the data to get it ready for graphing
results = results.reset_index().melt(id_vars=['month','day'], var_name=['hour','stat'])
results.head(5)

,month,day,hour,stat,value
0,1,weekday,0,min,0.04094
1,1,weekend,0,min,0.04094
2,2,weekday,0,min,0.04094
3,2,weekend,0,min,0.04094
4,3,weekday,0,min,0.04094


In [275]:
color_map = {'min':'lightskyblue',
             '25%':'royalblue',
             '50%':'black',
             '75%':'royalblue',
             'max':'lightskyblue'}

weekday_results_fig = px.line(results[results['day'] == 'weekday'], x='hour', y='value', color='stat', color_discrete_map=color_map, facet_col='month', facet_col_wrap=3, height=800, template='plotly_white', title='Distribution of Weekday Rates')
weekday_results_fig.show()

In [276]:
weekend_results_fig = px.line(results[results['day'] == 'weekend'], x='hour', y='value', color='stat', color_discrete_map=color_map, facet_col='month', facet_col_wrap=3, height=800, template='plotly_white', title='Distribution of Weekend Rates')
weekend_results_fig.show()